In [3]:
import os
access_token = os.environ["ACCESS_TOKEN"]
account_id_demo = os.environ["ACCOUNT_DEMO"]
account_id_hedge = os.environ["ACCOUNT_HEDGE"]
import oandapyV20
import oandapyV20.endpoints.transactions as trans
import pandas as pd

In [140]:
client = oandapyV20.API(access_token=access_token)
param = { "id": 900 }
r = trans.TransactionsSinceID(accountID=account_id_demo,  params=param)
trades= client.request(r)
param1 = { "id": 0 }
request_hedge = trans.TransactionsSinceID(accountID=account_id_hedge,  params=param1)
trades_hedge= client.request(request_hedge)

In [142]:
all_transactions_hedge = pd.DataFrame(trades_hedge['transactions'])
all_transactions = pd.DataFrame(trades['transactions'])
all_transactions = pd.concat([all_transactions_hedge,all_transactions])
all_transactions['time'] = pd.to_datetime(all_transactions['time'])
all_transactions['date'] = all_transactions['time'].dt.date
all_transactions = all_transactions[all_transactions['date'] == all_transactions['date'].max()]
all_transactions['pl'] = pd.to_numeric(all_transactions['pl'])
print(all_transactions[all_transactions['reason'].isin(['TRAILING_STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby(['accountID','reason'])['pl'].sum())
print(all_transactions['pl'].sum())

accountID             reason                  
101-004-31059296-001  TAKE_PROFIT_ORDER           5260.1565
                      TRAILING_STOP_LOSS_ORDER   -3975.0291
101-004-31059296-003  TAKE_PROFIT_ORDER           1291.0999
                      TRAILING_STOP_LOSS_ORDER   -9613.3486
Name: pl, dtype: float64
-5604.8283


In [144]:
pd.to_numeric(all_transactions['halfSpreadCost']).sum()

4605.7275

In [49]:
all_transactions_hedge['id'] = pd.to_numeric(all_transactions_hedge['id'])
all_transactions_hedge[(all_transactions_hedge['id'] >= 165) & (all_transactions_hedge['id'] < 171) ]

,id,accountID,userID,batchID,requestID,time,type,divisionID,siteID,accountUserID,...,triggerMode,tradesClosed,closedTradeID,tradeCloseTransactionID,distance,rejectReason,replacesOrderID,intendedReplacesOrderID,replacedByOrderID,cancellingTransactionID
164,165,101-004-31059296-003,31059296,165,115401766702038974,2025-04-08T14:11:06.723965571Z,MARKET_ORDER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,166,101-004-31059296-003,31059296,165,115401766702038974,2025-04-08T14:11:06.723965571Z,ORDER_FILL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,167,101-004-31059296-003,31059296,165,115401766702038974,2025-04-08T14:11:06.723965571Z,TAKE_PROFIT_ORDER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,168,101-004-31059296-003,31059296,165,115401766702038974,2025-04-08T14:11:06.723965571Z,TRAILING_STOP_LOSS_ORDER,NaN,NaN,NaN,...,TOP_OF_BOOK,NaN,NaN,NaN,0.00060,NaN,NaN,NaN,NaN,NaN
168,169,101-004-31059296-003,31059296,169,NaN,2025-04-08T14:12:32.787007486Z,ORDER_FILL,NaN,NaN,NaN,...,NaN,"[{'tradeID': '166', 'units': '1000000', 'reali...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,170,101-004-31059296-003,31059296,169,NaN,2025-04-08T14:12:32.787007486Z,ORDER_CANCEL,NaN,NaN,NaN,...,NaN,NaN,166,169,NaN,NaN,NaN,NaN,NaN,NaN


In [328]:
orders = pd.read_csv('/work/data/orders/all_orders.csv')
orders.rename(columns={'Type':'TypeB'},inplace=True)
orders['Time'] = pd.to_datetime(orders['Time'])
all_transactions['id'] = pd.to_numeric(all_transactions['id'])
all_transactions['batchID'] = pd.to_numeric(all_transactions['batchID'])
all_transactions = pd.merge(all_transactions,orders,left_on='batchID', right_on = 'Order_ID',how='left')

In [332]:
trans_orders = _dntk.execute_sql(
  'WITH trading_orders AS (\n    SELECT \n        CAST(all1.batchID AS INTEGER) as batchID,base.accountID,all1.Strategy,all1.TypeB,\n        base.pl,\n        date_trunc(\'minute\',all1.time) time_tf,\n        date_trunc(\'minute\', all1.time - INTERVAL \'6 minutes\') AS order_time\n    FROM all_transactions base \n    LEFT JOIN all_transactions order_filled \n        ON CAST(base.orderID AS INTEGER) = order_filled.id\n    LEFT JOIN all_transactions all1 \n        ON order_filled.batchID = all1.id\n    WHERE base.reason IN (\'TAKE_PROFIT_ORDER\', \'TRAILING_STOP_LOSS_ORDER\')\n)\nSELECT * FROM trading_orders\nORDER BY batchID ',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
trans_orders

,batchID,accountID,Strategy,TypeB,pl,time_tf,order_time
0,64,101-004-31059296-003,volume_based_hedge,BUY,889.4796,2025-04-08 10:21:00+00:00,2025-04-08 10:15:00+00:00
1,70,101-004-31059296-003,volume_based_hedge,BUY,-425.3835,2025-04-08 10:36:00+00:00,2025-04-08 10:30:00+00:00
2,76,101-004-31059296-003,volume_based_hedge,BUY,-456.8629,2025-04-08 10:46:00+00:00,2025-04-08 10:40:00+00:00
3,83,101-004-31059296-003,volume_based_hedge,BUY,31.1602,2025-04-08 11:16:00+00:00,2025-04-08 11:10:00+00:00
4,87,101-004-31059296-003,volume_based_hedge,BUY,-298.9971,2025-04-08 11:21:00+00:00,2025-04-08 11:15:00+00:00
...,...,...,...,...,...,...,...
56,1214,101-004-31059296-001,volume_based,BUY,-361.7824,2025-04-08 16:51:00+00:00,2025-04-08 16:45:00+00:00
57,1220,101-004-31059296-001,volume_based,BUY,233.6494,2025-04-08 16:56:00+00:00,2025-04-08 16:50:00+00:00
58,1228,101-004-31059296-001,volume_based,SELL,264.7012,2025-04-08 17:36:00+00:00,2025-04-08 17:30:00+00:00
59,1232,101-004-31059296-001,volume_based,SELL,-338.1336,2025-04-08 17:41:00+00:00,2025-04-08 17:35:00+00:00


In [338]:
strategy_agg=pd.merge(    
    trans_orders[trans_orders['accountID']=='101-004-31059296-003'],
    trans_orders[trans_orders['accountID']=='101-004-31059296-001'],
    on = 'time_tf', how='left')

In [359]:
prediction_actual.head()

,datetime,upward_move_6,downward_move_6,upward_move_20,downward_move_20,bearish_move_flag_20,bearish_move_flag_20.1,y_actual
0,2025-04-03 06:20:00+00:00,51.8,0.6,53.1,0.6,0,0,1
1,2025-04-03 06:25:00+00:00,41.5,6.9,41.6,6.9,0,0,1
2,2025-04-03 06:30:00+00:00,25.9,4.1,25.9,21.6,2,2,3
3,2025-04-03 06:35:00+00:00,12.3,7.2,12.3,35.2,2,2,2
4,2025-04-03 06:40:00+00:00,12.6,8.3,16.3,34.9,2,2,2


In [348]:
strategy_agg['pl_x'] = pd.to_numeric(strategy_agg['pl_x'])
strategy_agg['pl_y'] = pd.to_numeric(strategy_agg['pl_y'])
strategy_agg['pl_agg'] = strategy_agg['pl_x'] + strategy_agg['pl_y']
strategy_agg = strategy_agg[(strategy_agg['time_tf'].dt.date == strategy_agg['time_tf'].dt.date.max())
            ][['Strategy_x','pl_x','pl_y','pl_agg','order_time_x','TypeB_x']]

In [369]:
prediction_actual = pd.read_csv('/work/data/predictions/predictions_actuals.csv')
strategy_agg['order_time_x'] = pd.to_datetime(strategy_agg['order_time_x'])
prediction_actual['datetime'] = pd.to_datetime(prediction_actual['datetime'])
strategy_w_predictions = pd.merge(strategy_agg,prediction_actual,left_on='order_time_x',right_on = 'datetime')

In [354]:
orders[(orders['Time'].dt.date == orders['Time'].dt.date.max()) 
        & (orders['Account_ID'] == '101-004-31059296-003')][['Order_ID','datetime']].head()

KeyError: "['datetime'] not in index"

In [371]:
strategy_w_predictions.to_csv('/work/strategy.csv')

In [387]:
from Moduled_functions import get_data
access_token = os.environ["ACCESS_TOKEN"]
account_id_demo = os.environ["ACCOUNT_DEMO"]

base_data, current_price = get_data(100, access_token) 
print(f"Max available data {base_data['datetime'].max()}")

Max available data 2025-04-09 01:35:00+00:00


In [377]:
import schedule
import time
from datetime import datetime, timedelta
from Moduled_functions import get_data
access_token = os.environ["ACCESS_TOKEN"]
account_id_demo = os.environ["ACCOUNT_DEMO"]

def job():
    from Moduled_functions import get_data
    access_token = os.environ["ACCESS_TOKEN"]
    account_id_demo = os.environ["ACCOUNT_DEMO"]

    base_data, current_price = get_data(100, access_token) 
    print(f"Max available data {base_data['datetime'].max()}")
    print(f"Running task at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

def schedule_next_runs():
    now = datetime.now()
    
    # Calculate next XX:00:30 and XX:05:30 times
    current_minute = now.minute
    current_second = now.second
    
    # For XX:00:30
    if current_minute % 5 == 0 and current_second < 30:
        wait_seconds = 30 - current_second
    else:
        minutes_until_next = 5 - (current_minute % 5)
        wait_seconds = minutes_until_next * 60 + (30 - current_second)
        if wait_seconds < 0:
            wait_seconds += 300  # Add 5 minutes if we're past the target second
    
    print(f"Next run scheduled in {wait_seconds} seconds")
    return wait_seconds

# Run the scheduler
print("Scheduler started. Press Ctrl+C to exit.")
try:
    while True:
        wait_time = schedule_next_runs()
        time.sleep(wait_time)
        job()
        time.sleep(1)  # Small delay to prevent tight loop
except KeyboardInterrupt:
    print("\nScheduler stopped.")

Scheduler started. Press Ctrl+C to exit.
Next run scheduled in 183 seconds
Running task at 2025-04-09 01:35:30
Next run scheduled in 299 seconds

Scheduler stopped.


This scheduler will run every 5 minutes (at XX:00:30 and XX:05:30) and execute the defined job. The job currently just prints the current timestamp, but you can modify the job() function to include any tasks you want to run at these intervals. The scheduler will continue running until interrupted with Ctrl+C.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a127aa9e-0a77-4af9-a6ce-85e7a9b74042' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>